<h1>E-Commerce Actual Data Transactions from UK Retailer</h1>
<br>
In this notebook we are going to explore an <a href='https://archive.ics.uci.edu/ml/datasets/Online+Retail'>e-commerce dataset transactions from an UK retailer</a>, this dataset lists purchases made by approximately 40000 customers through a period of time of one year <i>(from 12/01/2010 to 12/09/2011)</i>. The main aim of this notebook is to develop a machine learning model that allows to anticipate the purhcases that will be made by a new customer, over the next year according to its firsts purchases.
<br>
<br>
This notebooks is divided by the following steps:
<ul>
    <li>Data Cleaning.</li>
    <li>Feature Exploration.</li>
    <li>Understanding Product Categories.</li>
    <li>Customers Categories.</li>
    <li>Classifying Customers.</li>
    <li>Testing Predictions.</li>
    <li>Explaining The Decissions of The Model.</li>
</ul>

<h2>Importing Necessary Packages, Modules and Libraries</h2>

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, nltk, warnings
import matplotlib.cm as cm
import itertools
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import preprocessing, model_selection, metrics, feature_selection
from sklearn.model_selection import GridSearchCV, learning_curve
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import neighbors, linear_model, svm, tree, ensemble
from wordcloud import WordCloud, STOPWORDS
from sklearn.ensemble import AdaBoostClassifier
from sklearn.decomposition import PCA
from IPython.display import display, HTML
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode,iplot
warnings.filterwarnings("ignore")
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline

c:\users\kleye\anaconda3\envs\ai\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<h2>Data Cleaning</h2>
<br>
Let's load the dataset into memory! with <code>encoding</code> parameter and the value of <code>ISO-8859-1</code> will allows us to be able to read the dataset, for a better understanding of this parameter and its value, check the next links:
<ul>
    <li><code>encoding</code>: <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html'>Pandas official docs for <code>read_csv</a> method.</li>
    <li><code>ISO-8859-1</code>: <a href=''>Codec registry and base classes</a>.</li>
</ul>
with <code>dtype</code> parameter by using a dictionary we convert into <code>string</code> variables the columns <code>CustomerID</code> and <code>InvoiceID</code>.

In [2]:
df = pd.read_csv('ecommerce-data/data.csv', encoding='ISO-8859-1', dtype={'CustomerID': str, 'InvoiceID': str})

Now with the data in memory, let's take a look a its dimensions.

In [3]:
df.shape

(541909, 8)

<b>541909</b> rows and <b>8</b> columns! Let's see general information of this dataset.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


Looks like there are null values, missing values. We must clean up this dataset!
<br>
<br>
Let's create a variable <code>columns_info</code> that will hold the data types of every column in the dataset. This is done by creating a <code>DataFrame</code> object that has information the types of the columns that the dataset <code>df</code> holds, this is obtained by passing <code>df.dtypes</code> as parameter for the dataset, then with the function <code>T</code> we <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.T.html'>transposed</a> our dataset and by renaming with the function <code>rename</code> we give a name to the new columns

In [5]:
columns_info= pd.DataFrame(df.dtypes).T.rename(index={0: 'Column Type'})

Looking for how many nulls we have in every single column of this dataset.
<br>
<br>
We append to <codde>columns_info</code> another <i>row in the index column</i>, called <code>Null Values (NV)</code>, this new row will hold the amount of null values for every column in the dataset, this is done thanks to <code>df.isnull().sum()</code> that is passed as the value for the parameter <code>data</code> in the constructor of <code>DataFrame</code>.

In [6]:
columns_info = columns_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'Null Values (NV)'}))

<b>How much do these null values composed the dataset?</b>
<br>
<br>
Again by appending to <code>columns_info</code> another <i>row in the index column</i>, called <code>Null Values (%)</code>, this new row will hold the <b>percentage</b> of null values for every column in the dataset, this is done thanks to <code>df.isnull().sum()/df.shape[0]*100</code> that is passed as the value for the parameter data in the constructor of <code>DataFrame</code>.
<br>
<br>
<b>Explaining the operation</b> <code>df.isnull().sum()/df.shape[0]*100</code>:
<br>
With <code>df.isnull().sum()</code> we get the total amount of null values in every column and then by dividing for the number of rows gotten from <code>df.shape[0]</code> we multiply by <code>100</code> to known how much percentage these null values represent in the dataset.

In [7]:
columns_info=columns_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:'Null Values (%)'}))

Let's take a look at <code>columns_info</code>:

In [8]:
columns_info

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
Column Type,object,object,object,int64,object,float64,object,object
Null Values (NV),0,0,1454,0,0,0,135080,0
Null Values (%),0,0,0.268311,0,0,0,24.9267,0


There are null values in the columns <code>Description</code> and <code>CustomerID</code>, these null values represent a <b>0.26%</b> and <b>24.9267%</b> respectively.
<br>
<br>
Let's see a random sample from the dataset:

In [9]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
32112,539042,22421,LIPSTICK PEN FUSCHIA,24,12/15/2010 15:37,0.42,15062,United Kingdom
475026,576860,22560,TRADITIONAL MODELLING CLAY,6,11/16/2011 15:58,1.25,17602,United Kingdom
146954,549023,85099B,JUMBO BAG RED RETROSPOT,13,4/5/2011 16:27,2.46,NaN,United Kingdom
210400,555278,21876,POTTERING MUG,1,6/1/2011 17:33,3.29,NaN,United Kingdom
148166,549185,22722,SET OF 6 SPICE TINS PANTRY DESIGN,8,4/7/2011 9:35,3.95,18272,United Kingdom


Now we know that almost <b>25%</b> of the transactions are not assigned to a particular client and arround <b>0.27%</b> of the transactions descriptions are not specified, there are several ways to deal with missing values:
<ul>
    <li>Impute values for the <code>CustomerID</code> and <code>Description</code>, in this case it is impossible, does not 
        make sense. </li>
    <li>Apply clustering analysis and see patterns in those <i>unknown</i> clients and <i>unknown</i> description of 
        products. Once these patterns are detected we can assign a labels for them and use these labels as a generic 
        <i>CustomerID</i> and <i>Description</i>.</li>
    <li>Delete the rows where these missing values are found.</li>
<ul>

For simplicity we'll delete the missing values with the method <code>dropna()</code>, understanding its parameters:
<ul>
    <li><code>axis</code>: with the value of <code>0</code> to specify that we are interested in the missing <i>rows</i>.
    </li>
    <li><code>subset</code>: The name of the column, where the missing values.</li>
    <li><code>inplace</code>: To return a new dataset that does not have missing values.</li>
</ul>

In [10]:
df.dropna(axis = 0, subset = ['CustomerID'], inplace = True)

We have cleaned the dataset from missing values! Once again by using the same methods as we did to know how many null values were in the dataset and how much they composed the dataset, we take a look to check up if these values were deleted from the dataset.

In [11]:
columns_info=pd.DataFrame(df.dtypes).T.rename(index={0:'Column Type'})
columns_info=columns_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'Null Values (NB)'}))
columns_info=columns_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:'Null Values (%)'}))

The methods have been applied, now let's see the information:

In [12]:
columns_info

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
Column Type,object,object,object,int64,object,float64,object,object
Null Values (NB),0,0,0,0,0,0,0,0
Null Values (%),0,0,0,0,0,0,0,0


Nice! The missing values were deleted from the dataset. Now what are we going to do is to check for duplicated values, <i>duplicate values can cause generalization problems, duplicated values can biased the model.</i> This check is done the next way.

In [13]:
print('Quantity of duplicate values: {}'.format(df.duplicated().sum()))

Quantity of duplicate values: 5225


There are <b>5225</b> duplicated values, let's take a look at some of them.
<br>
First we get the indexes of 5 random duplicated values, these indexes are going to be hold in the <code>indexex</code> var, then with the <code>loc[]</code> function we found to print them.

In [14]:
indexes = df.duplicated().sample(5).index
print('Samples of duplicate values:\n{}'.format(df.loc[indexes]))

Samples of duplicate values:
       InvoiceNo StockCode                       Description  Quantity  \
367502    568895     22384           LUNCH BAG PINK POLKADOT         2   
250481    559034     22139  RETROSPOT TEA SET CERAMIC 11 PC          3   
213077    555518     22383            LUNCH BAG SUKI DESIGN         10   
342265    566849     22809           SET OF 6 T-LIGHTS SANTA         6   
295344    562782     20725           LUNCH BAG RED RETROSPOT        10   

            InvoiceDate  UnitPrice CustomerID         Country  
367502  9/29/2011 13:11       1.65      15356  United Kingdom  
250481   7/5/2011 14:56       4.95      13109  United Kingdom  
213077   6/5/2011 10:58       1.65      14891  United Kingdom  
342265  9/15/2011 12:03       2.95      16341  United Kingdom  
295344   8/9/2011 13:14       1.65      13280  United Kingdom  


Cool! Time to drop them from the dataset, with the funciton <code>drop_duplicates</code> and the value <code>True</code> for the parameter <code>inplace</code>, this wo need won't need to assign back to the <code>df</code> dataframe, because it is on the same copy.


In [15]:
df.drop_duplicates(inplace=True)

We have finished with cleaning the dataset, now we are going to work on <b>Feature Exploration</b> to <i>have a view and a opinion over the columns that the dataset is made of.</i>
<br>
<h2>Feature Exploration</h2>
<br>
In the <a href='https://archive.ics.uci.edu/ml/datasets/Online+Retail'>UCl Machine Learning Repository</a> we can find the explanation of every column, I'll list their explanation below:
<ul>
    <li><code>InvoiceNo</code>: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. 
        If this code starts with letter <b>'c'</b>, it indicates a cancellation.</li>
    <li><code>StockCode</code>: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct 
        product.</li>
    <li><code>Description</code>: Product (item) name. Nominal.</li>
    <li><code>Quantity</code>: The quantities of each product (item) per transaction. Numeric.</li>
    <li><code>InvoiceDate</code>: Invice Date and time. Numeric, the day and time when each transaction was generated.</li>
    <li><code>UnitPrice</code>: Unit price. Numeric, Product price per unit in sterling.</li>
    <li><code>CustomerID</code>: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
    </li>
    <li><code>Country</code>: Country name. Nominal, the name of the country where each customer resides.</li>
</ul>
Every column from this dataset have been explained, now we have understood what they do, how they are composed and what information they hold! Let's explore some of them.

<h4>Exploring the Country Column</h4>
<br>
We are going to look at the countries from which customers made their orders, by creating a temporal dataframe object called <code>temp</code> that holds the columns <code>CustomerID</code>, <code>InvoiceNo</code> and <code>Country</code> and group this new temporal dataframe by counting <code>CustomerID</code>, <code>InvoiceNo</code> and <code>Country</code> groups. <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html'>Look here to understand better the <code>groupby()</code> function</a>.
<br>
<br>
This will hold every customer that did a transaction and from the country they did it, it is going to be mainly group by the column <code>CustomerID</code>, thanks to the method <code>count()</code> that return a <code>DataFrame</code> object we are going to be able to display its information and a clearer way.

In [113]:
temp = df[['CustomerID', 'InvoiceNo', 'Country']].groupby(['CustomerID', 'InvoiceNo', 'Country']).count()

Them with <code>reset_index</code> and the parameter <code>drop</code> with its default value of <code>False</code>, we reset the index and we <i>do not try to insert index into dataframe columns.</i> This resets the index to the default integer index. <a href='https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reset_index.html'>Check here for more info.</a>

In [114]:
temp = temp.reset_index(drop=False)

Great! Let's see how many countries buy from the e-commerce retailer, this done with the <code>pandas</code> function <code>value_counts()</code> that returns the quantity of unique values.

In [115]:
countries = temp['Country'].value_counts()
print('Number of countries that purchase from the e-commerce retailer: {}'.format(len(countries)))

Number of countries that purchase from the e-commerce retailer: 37


Let's see how much purchases were done by every country, we can do it using the <code>numpy</code> function <code>unique</code> passing it the <code>Country</code> column and <code>True</code> as value for the <code>return_counts</code> that will return the number of frequencies for every different value from the column specified, it will return two arrays; one array with the names of the countries that appear in the column <code>Country</code> and the other array with the frequency quantity that they appear.

In [26]:
COUNTRY, COUNTRY_ORDERS = np.unique(temp['Country'], return_counts=True)

Then by converting this two arrays into a <code>DataFrame</code> object we sort its values by ascending order according to the column <code>Purchases</code>, this is done thanks to the <code>sort_values</code> and the current parameters and its values we used for. We then take a look at its first 10 rows. 

In [27]:
data_country_purchases = pd.DataFrame({'Country': COUNTRY, 'Orders': COUNTRY_ORDERS})
data_country_purchases.sort_values(by='Orders', ascending=False, inplace=True, ignore_index=True)
data_country_purchases.head(10)

,Country,Orders
0,United Kingdom,19857
1,Germany,603
2,France,458
3,EIRE,319
4,Belgium,119
5,Spain,105
6,Netherlands,101
7,Switzerland,71
8,Portugal,70
9,Australia,69


<h4>Using Plotly for Geo Chars by Country </h4>

<b>What is Plotly?</b>
<blockquote>
    The plotly Python library (plotly.py) is an interactive, open-source plotting library that supports over 40 unique 
    chart types covering a wide range of statistical, financial, geographic, scientific and 3-dimensional use-cases.
    <a href='https://plot.ly/python/getting-started/#overview><i>From Plot.ly</i></a>
</blockquote>

Here we are using <a href='https://plot.ly/python/reference/#choropleth'>Choropleth Maps</a> and its configurations and styling is <a href='https://plot.ly/python/map-configuration/'>here</a>.

In [94]:
data = dict(type='choropleth',
            locations=countries.index,
            locationmode='country names',
            z=countries,
            text=countries.index,
            colorbar={'title':'Quantity Orders'},
            colorscale = ["#8dd3c7", "#ffffb3", "#bebada", "#fb8072",
                          "#80b1d3", "#fdb462", "#b3de69", "#fccde5",
                          "#d9d9d9", "#bc80bd", "#ccebc5", "#ffed6f",
                          "#8dd3c7", "#ffffb3", "#bebada", "#fb8072",
                          "#80b1d3", "#fdb462", "#b3de69", "#fccde5",
                          "#d9d9d9", "#bc80bd", "#ccebc5", "#ffed6f",
                          "#8dd3c7", "#ffffb3", "#bebada", "#fb8072",
                          "#80b1d3", "#fdb462", "#b3de69", "#fccde5",
                          "#d9d9d9", "#bc80bd", "#ccebc5", "#ffed6f"],
            autocolorscale=True,
            reversescale=True)

In [149]:
layout = dict(title='Number of Orders Per Country',
              geo=dict(showframe=True,
                       projection={'type':'orthographic'}))

In [150]:
choromap = go.Figure(data=data,
                     layout=layout)
iplot(choromap, validate=True)

<h4>Customers, Transactions and Products</h4>
<br>
Let's take a look at how many customers, transactions and products this dataset has. This is done by creating a <code>DataFrame</code> object, we the by accessing the columns <code>StockCode</code> for <b>Products</b>, <code>InvoiceNo</code> for <b>Transactions</b> and <code>CustomerID</code> for <b>Customers</b> and by applying the function <code>value_counts()</code> we obtained how many unique values are for every of the specified columns.

In [101]:
pd.DataFrame([{'Products': len(df['StockCode'].value_counts()),    
               'Transactions': len(df['InvoiceNo'].value_counts()),
               'Customers': len(df['CustomerID'].value_counts())}],
             columns = ['Products', 'Transactions', 'Customers'], index = ['Quantity'])

,Products,Transactions,Customers
Quantity,3684,22190,4372


Interesting! the cleaned dataset tells us that <b>4372</b> customers made <b>22190</b> transactions for <b>3684</b> unique products. Let's take a look at how many times a product was purchased, this is done as we did it previously for guessing the countries and their transactions, but this time we focus on <code>StockCode</code>:

In [187]:
PRODUCTS_ID, QUANTITY_SOLD = np.unique(df['StockCode'], return_counts=True)

In [188]:
products_purchases = pd.DataFrame({'Product ID': PRODUCTS_ID, 'Quantity Sold': QUANTITY_SOLD})
products_purchases.sort_values(by='Quantity Sold', ascending=False, inplace=True, ignore_index=True)
products_purchases.head(10)

,Product ID,Quantity Sold
0,85123A,2065
1,22423,1894
2,85099B,1659
3,47566,1409
4,84879,1405
5,20725,1346
6,22720,1224
7,POST,1196
8,22197,1110
9,23203,1108


In [148]:
CUSTOMERS, CUSTOMERS_TRANSC = np.unique(df['CustomerID'], return_counts=True)
customers_transactions = pd.DataFrame({'Customer': CUSTOMERS, 'Quantity Transactions': CUSTOMERS_TRANSC})
customers_transactions.sort_values(by='Quantity Transactions', ascending=False, inplace=True, ignore_index=True)
customers_transactions.head(10)

,Customer,Quantity Transactions
0,17841,7812
1,14911,5898
2,14096,5128
3,12748,4459
4,14606,2759
5,15311,2478
6,14646,2085
7,13089,1853
8,13263,1667
9,14298,1640


In [22]:
!git add . e-commerce_uk_retailer_machine_learning_analysis.ipynb
!git commit -m "Data Preparation rename to Data Cleaning"
!git push origin master --force

The file will have its original line endings in your working directory


[master 7a172e2] Data Preparation rename to Data Cleaning
 1 file changed, 451 insertions(+), 28 deletions(-)


To https://github.com/kleyersoma/E-Commerce_UK_Retailer_ML
   bad1f38..7a172e2  master -> master
